In [1]:
%matplotlib inline

from collections import Counter
import glob
import itertools
from itertools import izip
import os

from IPython.core.display import HTML, Image
from matplotlib import gridspec
from matplotlib_venn import venn3, venn2
import numpy as np
import pandas as pd
import scipy
import seaborn as sns

import flotilla
from gscripts.general import dataviz
from gscripts.general import parsers
from gscripts.general import region_helpers
from gscripts import GO
from gscripts import miso


img_dir = "/nas3/gpratt/Dropbox/TAF15/Data/MN_data/GP_analysis_RNAseq/"

:0: FutureWarning: IPython widgets are experimental and may change in the future.


#Setup Basics

In [2]:
def counts_to_rpkm(featureCountsTable):
    counts = featureCountsTable.ix[:,5:]
    lengths = featureCountsTable['Length']
    mapped_reads = counts.sum()
    return (counts * pow(10,9)).div(mapped_reads, axis=1).div(lengths, axis=0)

def gencode_to_ensembl(gene_list):
    for gene in gene_list:
        yield gene.split(".")[0]
        
def plot_go_enrichment(df, filter_value=None, **kwargs):
    
    new_index = []
    for index, description in izip(df.index, df['GO Term Description']):
        new_index.append(list(index[:-1]) + [description])
    df.index = pd.MultiIndex.from_tuples(new_index)

    go_matrix = df['Bonferroni-corrected Hypergeometric p-Value'].apply(lambda x: -1 * np.log10(x))
    go_matrix = go_matrix.unstack(range(len(go_matrix.index.levels) - 1))
    go_matrix = go_matrix.fillna(0)
    if filter_value is not None:
        go_matrix = go_matrix[go_matrix.apply(max, axis=1) > filter_value]
        
    sns.clustermap(go_matrix, robust=True, **kwargs)

In [4]:
!mkdir -p /nas3/gpratt/projects/fet_family/analysis/human_mn_v2/
!scp -q  tscc-login.sdsc.edu:~/projects/fet_family/analysis/human_mn_v2/*.{final.out,metrics,rpkm} /nas3/gpratt/projects/fet_family/analysis/human_mn_v2/
!scp -q  tscc-login.sdsc.edu:~/projects/fet_family/analysis/human_mn_v1/miso_summary_filtered.csv /nas3/gpratt/projects/fet_family/analysis/human_mn_v1/

In [3]:
gene_id_to_name = region_helpers.gene_id_to_name("/nas3/gpratt/gencode/gencode.v17.annotation.gtf.db")
name_to_gene_id = {value: key for key, value in gene_id_to_name.items()}

#MN Read Mapping QC


In [4]:
analysis_dir = "/nas3/gpratt/projects/fet_family/analysis/human_mn_v2/"
fet_clip = parsers.rnaseq_metrics(analysis_dir)

filtered_guttman_clip = fet_clip[[ "Input Reads",
                            #"Reads After Triming",
                            "Uniquely Mapped Reads",
                            "Uniquely mapped reads %",
                            ]]

HTML(filtered_guttman_clip.to_html(formatters={"Input Reads" : parsers.commas,
                                     "Reads After Triming" : parsers.commas,
                                     "Reads after Quality Filtering" : parsers.commas,
                                     "Uniquely Mapped Reads" : parsers.commas,

                                     } ))

,Input Reads,Uniquely Mapped Reads,Uniquely mapped reads %
FUS_TAF15_shRNA_1,"48,453,046","23,319,915",93.21%
FUS_TAF15_shRNA_2,"42,856,351","35,500,133",94.19%
FUS_shRNA_1,"48,334,790","39,797,138",94.29%
FUS_shRNA_2,"46,737,651","38,856,955",94.29%
Scrm_1,"22,513,222","18,610,246",94.27%
Scrm_2,"33,675,077","27,513,846",94.05%
TAF15_TDP43_shRNA_1,"46,433,859","38,598,286",94.06%
TAF15_TDP43_shRNA_2,"38,472,528","32,216,211",94.29%
TAF15_shRNA_1,"47,587,547","39,865,790",94.46%
TAF15_shRNA_2,"39,768,330","33,071,863",94.36%


In [5]:
filtered_guttman_clip['Uniquely Mapped Reads'].mean()

32461185.916666668

In [6]:
np.mean([float(item[:-1]) for item in filtered_guttman_clip['Uniquely mapped reads %']])

94.143333333333331

#Load Counts

In [24]:
all_counts = pd.read_table("/nas3/gpratt/Dropbox/TAF15/Data/MN_data/GP_analysis_RNAseq/all_counts.txt", skiprows=1, index_col=0)
all_counts.columns = [os.path.basename(col).split(".")[0] for col in all_counts.columns]
rpkm = counts_to_rpkm(all_counts).T

#Make Metadata

In [25]:
metadata = []
for item in rpkm.index:
    experiment = item.split("_")[:-2]
    if len(experiment) == 0:
        experiment = item.split("_")[:-1]
    experiment = "_".join(experiment)
    replicate = item.split("_")[-1]
    metadata.append(["MN", experiment, replicate, item])

rpkm.index = pd.MultiIndex.from_tuples(metadata, names=['cell_type', "knockdown", "rep", 'exp'])
    
#metadata = pd.DataFrame(metadata).T
#metadata.index.name = "exp"
#metadata.to_csv("/nas3/gpratt/Dropbox/TAF15/Data/MN_data/GP_analysis_RNAseq/sampleInfo.csv")

In [26]:
rpkm

Geneid                                         ENSG00000223972.4  \
cell_type knockdown   rep exp                                      
MN        FUS         1   FUS_shRNA_1                   0.348521   
                      2   FUS_shRNA_2                   0.341640   
          FUS_TAF15   1   FUS_TAF15_shRNA_1             0.477444   
                      2   FUS_TAF15_shRNA_2             0.610676   
          Scrm        1   Scrm_1                        0.239699   
                      2   Scrm_2                        0.286792   
          TAF15       1   TAF15_shRNA_1                 0.364871   
                      2   TAF15_shRNA_2                 0.292417   
          TAF15_TDP43 1   TAF15_TDP43_shRNA_1           0.557530   
                      2   TAF15_TDP43_shRNA_2           0.336254   
          TDP43       1   TDP43_shRNA_1                 0.352274   
                      2   TDP43_shRNA_2                 0.511942   

Geneid                                         ENSG00000227232.4  \
cell_type knockdown   rep exp                                      
MN        FUS         1   FUS_shRNA_1                  38.587782   
                      2   FUS_shRNA_2                  36.515108   
          FUS_TAF15   1   FUS_TAF15_shRNA_1            38.657118   
                      2   FUS_TAF15_shRNA_2            38.052035   
          Scrm        1   Scrm_1                       31.471945   
                      2   Scrm_2                       34.302594   
          TAF15       1   TAF15_shRNA_1                34.805379   
                      2   TAF15_shRNA_2                35.214815   
          TAF15_TDP43 1   TAF15_TDP43_shRNA_1          33.610143   
                      2   TAF15_TDP43_shRNA_2          36.152039   
          TDP43       1   TDP43_shRNA_1                38.363623   
                      2   TDP43_shRNA_2                38.459892   

Geneid                                         ENSG00000243485.1  \
cell_type knockdown   rep exp                                      
MN        FUS         1   FUS_shRNA_1                   2.891300   
                      2   FUS_shRNA_2                   1.866434   
          FUS_TAF15   1   FUS_TAF15_shRNA_1             2.326583   
                      2   FUS_TAF15_shRNA_2             2.772769   
          Scrm        1   Scrm_1                        1.855145   
                      2   Scrm_2                        1.972994   
          TAF15       1   TAF15_shRNA_1                 1.603703   
                      2   TAF15_shRNA_2                 2.305062   
          TAF15_TDP43 1   TAF15_TDP43_shRNA_1           2.477124   
                      2   TAF15_TDP43_shRNA_2           2.313267   
          TDP43       1   TDP43_shRNA_1                 1.893341   
                      2   TDP43_shRNA_2                 2.586408   

Geneid                                         ENSG00000237613.2  \
cell_type knockdown   rep exp                                      
MN        FUS         1   FUS_shRNA_1                   0.059065   
                      2   FUS_shRNA_2                   0.028949   
          FUS_TAF15   1   FUS_TAF15_shRNA_1             0.000000   
                      2   FUS_TAF15_shRNA_2             0.140751   
          Scrm        1   Scrm_1                        0.000000   
                      2   Scrm_2                        0.041313   
          TAF15       1   TAF15_shRNA_1                 0.029200   
                      2   TAF15_shRNA_2                 0.000000   
          TAF15_TDP43 1   TAF15_TDP43_shRNA_1           0.100392   
                      2   TAF15_TDP43_shRNA_2           0.223561   
          TDP43       1   TDP43_shRNA_1                 0.031716   
                      2   TDP43_shRNA_2                 0.046092   

Geneid                                         ENSG00000268020.1  \
cell_type knockdown   rep exp                                      
MN        FUS         1   FUS_shRNA_1               

In [27]:
result = {}
for item in rpkm.index: 
    result[item[-1]] = {key: value for value, key in zip(item, rpkm.index.names)}
sampleInfo = pd.DataFrame(result).T 
sampleInfo.to_csv("/nas3/gpratt/Dropbox/TAF15/Data/MN_data/GP_analysis_RNAseq/sampleInfo.csv")

all_counts.T.ix[sampleInfo.index].T.to_csv("/nas3/gpratt/Dropbox/TAF15/Data/MN_data/GP_analysis_RNAseq/important_counts.csv")

In [28]:
sampleInfo

,cell_type,exp,knockdown,rep
FUS_TAF15_shRNA_1,MN,FUS_TAF15_shRNA_1,FUS_TAF15,1
FUS_TAF15_shRNA_2,MN,FUS_TAF15_shRNA_2,FUS_TAF15,2
FUS_shRNA_1,MN,FUS_shRNA_1,FUS,1
FUS_shRNA_2,MN,FUS_shRNA_2,FUS,2
Scrm_1,MN,Scrm_1,Scrm,1
Scrm_2,MN,Scrm_2,Scrm,2
TAF15_TDP43_shRNA_1,MN,TAF15_TDP43_shRNA_1,TAF15_TDP43,1
TAF15_TDP43_shRNA_2,MN,TAF15_TDP43_shRNA_2,TAF15_TDP43,2
TAF15_shRNA_1,MN,TAF15_shRNA_1,TAF15,1
TAF15_shRNA_2,MN,TAF15_shRNA_2,TAF15,2
